# Collecting NDVI Data for Melbourne and Christchurch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**This note has been configured to be run once in order**

This notebook will use Google Earth Engine and geemap to source the data we require for assessing greenspace in our two cities. Before starting data collection, it is important to understand the reasoning behind collecting it.

NDVI data will help to understand the quality of greenspaces within a city. As our report focuses on liveability, we want to show the appeal of these greenspaces for people living in the city. Access to greenspace can provide a shared space for activity, providing physical and mental benefits to health. This means we will look at the vegetation quality during months when greenspaces are most often utilised, eg. spring/summer. We will then use the median reducer to get an overall picture of vegetation health during this time.

In the warmer months, greenspaces can also mitigatethe urban heath island effect.


The data we collect should be recent enough to be relevant today, but also be show the general trend of greenspace. This means in our image collection, we will source imagery from the last five years.

To ensure that the data is of the highest quality, a cloudy pixel percentage threshold of 5 percent or lower will be applied.

Once this data is collected, we will be able to compare it to land use data from OSM to determine the quality and accessiblity of greenspace in each city.

In [ ]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 1.6 MB/s eta 0:00:00


In [8]:
# Importing relevant packages
import geemap
import ee
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [10]:
# Authenticating Google Earth Engine
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=C5ZF8IM42gXZ7BrSjQpjHHBec0AIxONLVWlVT2eaGMM&tc=F7znQU6D6IL-w-LHvRx9PSrMqQHQ8SW1XF1GcavOg0Q&cc=9eIOQ9yvJjcN7_PhJlXWOeJMC3kQ7K-f5hNhdLsTHqM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXl19l_3tHDYl5N7JWvsX-1qG4geiOqO85vtHrsqnRxHwQnenyTGuv0

Successfully saved authorization token.


## Melbourne

Since we want to keep a consistent method of analysis, we can use OpenStreetMaps to find a bounding bbox to clip the NDVI image to.

We will use the City of Melbourne rather than Greater Melbourne as the resultant image would be too large otherwise.

In [ ]:
!pip install osmnx

In [ ]:
import osmnx as ox

# setting place name
melb = 'City of Melbourne, Australia'

# creating a geodataframe
melb = ox.geocode_to_gdf(melb)

# getting bounding box of melbourne
bounds = list(melb.bounds.iloc[0])
bounds

# using bounds from OSM to create a ee.geometry
aoi = ee.Geometry.BBox(bounds[0], bounds[1], bounds[2], bounds[3])

### Getting the Image Collection

Parameters for the ImageCollections for both Melbourne and Christchurch:
- year period for enough range, that is still relevant to today
- from the sentinel-2 satellite
- Bounded by the area of interest
- Images taken between september and feburary (spring/summer)
- cloudy pixel percentage less than 5 percent

In [ ]:
# Choosing start and end date (last 3 years of data)
start_date = '2018-10-10'
end_date = '2023-10-10'

# Sourcing imagery from sentinel-2
collection = ee.ImageCollection("COPERNICUS/S2_SR")\
    .filterDate(start_date, end_date)\
    .filterBounds(aoi)\
    .filter(ee.Filter.calendarRange(9, 2, 'month'))\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))\
    .sort('CLOUDY_PIXEL_PERCENTAGE')

In [15]:
# Calculating and adding an NDVI band to each image
def compute_ndvi(img):
    # Calculate NDVI by normalizing the difference between near-infrared (B5) and red (B4) bands
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Add the computed NDVI band to the input image
    return img.addBands(ndvi)

collection = collection.map(compute_ndvi)

In [ ]:
collection

This ImageCollection is quite large, so we will apply a reducer to find the median values of each band.

In [ ]:
# applying median reducer
median = collection.reduce(ee.Reducer.median())

In [19]:
# Visualisation of median NDVI over Melbourne City
Map = geemap.Map(center = [-37.8136, 144.9631], zoom=12)

vis = {'bands': 'NDVI_median',
       min:-1,
       max:1,
       'palette':['#8bc4f9','#c9995c', '#c7d270', '#8add60', '#097210']}

Map.add_ee_layer(median, vis)
Map

Map(center=[-37.8136, 144.9631], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

We can already see the presence of greenspace in this image, so now we will crop the image to just our area of interest.

### Cropping the Image

In [ ]:
# Clipping the median image to the roi specified earlier, and reducing image size
median_clipped = median.clipToBoundsAndScale(aoi, scale=10)

In [ ]:
median_clipped

In [ ]:
# Visualisation of clipped image
Map = geemap.Map(center = [-37.8136, 144.9631], zoom=12)

# using vis params from earlier
Map.add_ee_layer(median_clipped, vis)
Map

Map(center=[-37.8136, 144.9631], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

### Exporting the final image

In [ ]:
# checking projection info
projection = median_clipped.select('NDVI_median').projection().getInfo()

print(projection)

# setting crs and other information
crs = projection['crs']
crs_transform = projection['transform']
description = 'export_S2_chc'
scale=1

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [8.983152841195215e-05, 0, 0, 0, -8.983152841195215e-05, 0]}


In [ ]:
# selecting on the NDVI band
ndvi = median_clipped.select('NDVI_median')
ndvi

In [ ]:
!pwd

In [ ]:
# exporting final median ndvi to a GeoTIFF file with set crs
geemap.ee_export_image(ndvi, '/content/drive/MyDrive/FinalProject/Data/NDVI-imagery/NDVI_melbourne.tif', crs=crs, crs_transform=crs_transform)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/FinalProject/Data/NDVI-imagery/NDVI_melbourne.tif


## Christchurch

The image producing when using OSM to define the bounding box of Chirstchurch is slightly too large to export from GEE. THis means we will use an alternative bounding box to clip the image with that excludes the Waimakariri River in the north. As our analysis is focusing on city greenspace, it will be ok to crop out some of the agricultural land in this area.

In [11]:
# getting bounding box of Christchurch through user-drawn polygon
bounds = [172.4764, -43.5858, 172.7648, -43.4523]
bounds

# using bounds from OSM to create a ee.geometry
chc_aoi = ee.Geometry.BBox(bounds[0], bounds[1], bounds[2], bounds[3])

### Getting the Image Collection

In [12]:
# Choosing start and end date (last 3 years of data)
start_date = '2018-10-10'
end_date = '2023-10-10'

# Sourcing imagery from sentinel-2
collection = ee.ImageCollection("COPERNICUS/S2_SR")\
    .filterDate(start_date, end_date)\
    .filterBounds(chc_aoi)\
    .filter(ee.Filter.calendarRange(9, 2, 'month'))\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))\
    .sort('CLOUDY_PIXEL_PERCENTAGE')

In [16]:
collection = collection.map(compute_ndvi)
collection

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [17]:
# applying median reducer
median = collection.reduce(ee.Reducer.median())

In [20]:
# Visualisation of median NDVI over Chirstchurch City
Map = geemap.Map(center = [-43.5876073185094, 172.725385802955], zoom=10)

# visualising NDVI band
Map.add_ee_layer(median, vis)
Map

Map(center=[-43.5876073185094, 172.725385802955], controls=(WidgetControl(options=['position', 'transparent_bg…

### Cropping the Image

In [21]:
# Clipping the median image to the roi specified earlier, and reducing image size
median_clipped = median.clipToBoundsAndScale(chc_aoi, scale=10)
median_clipped

In [22]:
# Visualisation of clipped image
Map = geemap.Map(center = [-43.5876073185094, 172.725385802955], zoom=10)

# using vis params from earlier
Map.add_ee_layer(median_clipped, vis)
Map

Map(center=[-43.5876073185094, 172.725385802955], controls=(WidgetControl(options=['position', 'transparent_bg…

### Exporting the Final Image

In [27]:
# checking projection info
projection = median_clipped.select('NDVI_median').projection().getInfo()

print(projection)

# setting crs and other information
chc_crs = projection['crs']
chc_crs_transform = projection['transform']
description = 'export_S2_chc'
scale=1

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [8.983152841195215e-05, 0, 0, 0, -8.983152841195215e-05, 0]}


In [24]:
# selecting on the NDVI band
ndvi = median_clipped.select('NDVI_median')
ndvi

In [26]:
ndvi.select('NDVI_median').projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:4326',
 'transform': [8.983152841195215e-05, 0, 0, 0, -8.983152841195215e-05, 0]}

In [28]:
# exporting final median ndvi to a GeoTIFF file with set crs
geemap.ee_export_image(ndvi, '/content/drive/MyDrive/FinalProject/Data/NDVI-imagery/NDVI_christchurch.tif', crs=chc_crs, crs_transform=chc_crs_transform)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/FinalProject/Data/NDVI-imagery/NDVI_christchurch.tif
